In [1]:
import json
import sqlite3
import os
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import mysql.connector

# Specify the full path to the database file
db_path = 'C:/sqlite3/auction.db'

# Create a connection to the SQLite database
# If the database does not exist, it will be created automatically
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Create the Items table if it does not exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Items (
        item_id INT PRIMARY KEY,
        item_name TEXT,
        quality TEXT,
        item_level INT,
        required_level INT,
        item_class TEXT,
        item_subclass TEXT,
        purchase_price_gold INT,
        purchase_price_silver INT,
        sell_price_gold INT,
        sell_price_silver INT,
        max_count INT,
        is_stackable INT
    )
''')

# Create the Auctions table if it does not exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Auctions (
        auction_id INT PRIMARY KEY,
        bid INT,
        buyout INT,
        quantity INT,
        time_left TEXT,
        item_id INT
    )
''')

# Create the ActionEvents table if it does not exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS ActionEvents (
        auction_id INT,
        record DATETIME,
        PRIMARY KEY (auction_id, record),
        FOREIGN KEY (auction_id) REFERENCES Auctions(auction_id)
    )
''')

# Confirm the changes and close the connection
conn.commit()
conn.close()

# Load MySQL database configuration from a JSON file
with open('C:/Users/CMIRANDA/Documents/BOX/auctiondblocal/migrator/config.json') as f:  # Replace with the path to your JSON file
    config = json.load(f)

# Connect to the MySQL database
mysql_db = mysql.connector.connect(**config['database'])

# Extract data from MySQL
query = "SELECT * FROM Items"  # replace with your SQL query
df = pd.read_sql_query(query, mysql_db)

# Connect again to the SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Delete all records from the Items table
cursor.execute("DELETE FROM Items")
conn.commit()

# Import data from the DataFrame to the Items table
df.to_sql('Items', conn, if_exists='append', index=False)

# Print a message indicating that the data has been successfully imported into the Items table
print("The data has been successfully imported into the Items table.")

# Close the connection to the database
conn.close()

# Define the path to the directory containing the JSON files
data_dir = 'C:/Users/CMIRANDA/Documents/BOX/auctiondblocal/migrator'  # Replace this with the path to your JSON files

# Connect to the SQLite database
db = sqlite3.connect(db_path)
cursor = db.cursor()

# Iterate through all JSON files in the data directory and its subdirectories
for root, dirs, files in os.walk(data_dir):
    for filename in tqdm(files):
        if filename.endswith(".json") and filename != 'config.json':
            filepath = os.path.join(root, filename)
            try:
                data = json.load(open(filepath, "r"))
            except (FileNotFoundError, json.JSONDecodeError) as e:
                print(f"Error reading file {filepath}: {e}")
                continue

            print(f"Processing file: {filepath}")

            # Extract the timestamp of the auction from the filename
            auction_record = datetime.strptime(filename[:-5], "%Y%m%dT%H")

            # Prepare auction data for insertion
            auctions_data = [
                (auction["id"], auction["bid"], auction["buyout"], auction["quantity"], auction["time_left"], auction["item"]["id"])
                for auction in data["auctions"]
            ]
            action_events_data = [(auction["id"], auction_record.strftime('%Y-%m-%d %H:%M:%S')) for auction in data["auctions"]]

            # Insert auction data into the Auctions table
            try:
                cursor.executemany("""
                    INSERT OR REPLACE INTO Auctions (auction_id, bid, buyout, quantity, time_left, item_id)
                    VALUES (?, ?, ?, ?, ?, ?)
                """, auctions_data)
                db.commit()
                print(f"Auction data from file {filepath} successfully inserted into Auctions.")
            except sqlite3.Error as err:
                db.rollback()
                print(f"Error inserting auction data for file {filepath} in Auctions: {err}")

            # Insert data into ActionEvents
            try:
                cursor.executemany("""
                    INSERT OR REPLACE INTO ActionEvents (auction_id, record)
                    VALUES (?, ?)
                """, action_events_data)
                db.commit()
                print(f"Auction events for file {filepath} successfully inserted in ActionEvents.")
            except sqlite3.Error as err:
                db.rollback()
                print(f"Error inserting auction events for file {filepath} in ActionEvents: {err}")

# Close the cursor and the database connection
cursor.close()
db.close()


C:\Users\CMIRANDA\AppData\Local\Temp\ipykernel_10924\3355335318.py:71: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, mysql_db)


The data has been successfully imported into the Items table.


100%|██████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 4001.24it/s]
0it [00:00, ?it/s]
  0%|                                                                                           | 0/24 [00:00<?, ?it/s]

Processing file: C:/Users/CMIRANDA/Documents/BOX/auctiondblocal/migrator\auctions\09-01-2024\20240109T00.json
Auction data from file C:/Users/CMIRANDA/Documents/BOX/auctiondblocal/migrator\auctions\09-01-2024\20240109T00.json successfully inserted into Auctions.


  4%|███▍                                                                               | 1/24 [00:08<03:04,  8.01s/it]

...

 96%|██████████████████████████████████████████████████████████████████████████████▌   | 23/24 [04:08<00:10, 10.01s/it]

Auction events for file C:/Users/CMIRANDA/Documents/BOX/auctiondblocal/migrator\auctions\10-01-2024\20240110T22.json successfully inserted in ActionEvents.
Processing file: C:/Users/CMIRANDA/Documents/BOX/auctiondblocal/migrator\auctions\10-01-2024\20240110T23.json
Auction data from file C:/Users/CMIRANDA/Documents/BOX/auctiondblocal/migrator\auctions\10-01-2024\20240110T23.json successfully inserted into Auctions.


100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [04:18<00:00, 10.78s/it]


Auction events for file C:/Users/CMIRANDA/Documents/BOX/auctiondblocal/migrator\auctions\10-01-2024\20240110T23.json successfully inserted in ActionEvents.


100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1000.79it/s]
